In [20]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [21]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [22]:
%run -i ../../zkstats/core.py

In [33]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [34]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: arbitraryyy, just to make sure after filtered, it's not empty
dummy_data = np.random.uniform(1, 100, len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

# where(element > 30)
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
gt30_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 30].reshape(1,-1,1)
dummy_theory_output = torch.mean(gt30_dummy_data_tensor)

In [35]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # where part
        num_fil_X = torch.sum((X>30).double())
        fil_X = torch.tensor([ele[0] if ele[0]>30 else 0 for ele in X[0]])
        # fil_X = torch.where(X>30, X, 0)
        return (torch.abs(torch.sum(fil_X)-num_fil_X*(self.w))<=torch.abs(0.01*num_fil_X*self.w), self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_83676/2311067536.py:11: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  fil_X = torch.tensor([ele[0] if ele[0]>30 else 0 for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_83676/2311067536.py:11: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  fil_X = torch.tensor([ele[0] if ele[0]>30 else 0 for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_83676/2311067536.py:11: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use 

In [36]:
# prover calculates settings, send to verifier
gt30_data_tensor = data_tensor[data_tensor > 30].reshape(1,-1,1)
print("new tensor: ", gt30_data_tensor.size())
# gt30_data_tensor = torch.reshape(gt30_data_tensor_raw, (1, int(gt30_data_tensor_raw.size()[0]), 1))
theory_output = torch.mean(gt30_data_tensor)
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        
    def forward(self,X):
        # where part
        num_fil_X = torch.sum((X>30).double())
        fil_X = torch.tensor([ele[0] if ele[0]>30 else 0 for ele in X[0]])
        # fil_X = torch.where(X>30, X, 0)
        # print("fil_X: ", fil_X)
        return (torch.abs(torch.sum(fil_X)-num_fil_X*(self.w))<=torch.abs(0.01*num_fil_X*self.w), self.w)
    

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [2], "resources", settings_path)

new tensor:  torch.Size([1, 272, 1])
Theory_output:  tensor(52.3676)
==== Generate & Calibrate Setting ====


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_83676/1510789480.py:16: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  fil_X = torch.tensor([ele[0] if ele[0]>30 else 0 for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_83676/1510789480.py:16: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  fil_X = torch.tensor([ele[0] if ele[0]>30 else 0 for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_83676/1510789480.py:16: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use 

scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-160256,1254],"logrows":18,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":1516,"total_const_size":5,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [37]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 25.330917835235596 seconds
Theory output:  tensor(52.3676)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 52.25
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12572659313263335624, 14443766455855958404, 432930639589567449, 1881177029071802301], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18278699600166517679, 8643075271396760825, 7891176514265388517, 2236426588013702123]]], 'proof': '081b32cfa53a82a5e5e325fab4c025bafd3ffd877ab54dad1470d8b4abef97af2522f63b20599b39f35cffc313f6eb4c3686d5c65fd1cb11330fcc56467194172776ffe88ba29f73f22fbf1a124dc8f59f63f1331a7f65b67086f65104017a721f088a939e9ee96f7c214428ff53e4a5d9ad37436952c99bbbf5691a83eb389d0692475598c9ce2355d43f1f84ca9628b45f1c7dc1d31277c578f00016cae77210afe97e45126411c0894bec0b67d5bb4843e5a9c46fd81415e5cc8f0618f7e418519ac84eb119d8f6bec0061c870eb47485cdd10844a2b2aa24eaa4d8b9cd6f2769a9c783c41264f03462820254a4c38a2883fa85273d5239fe1a67e5f5914c1cba82f8f4aa4c5fe1acea94d570a9f03d3e2300f1776037b33b1a40a044c2c909461ca28ba4811b84686c24e5ea4bc4eb1cf0c86d8fb5518e72071d2d2a0738025f8c0eb3037213f2c2d4acb988fac25745899130cea698ab8c81633f1ce54

In [39]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[12572659313263335624, 14443766455855958404, 432930639589567449, 1881177029071802301], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18278699600166517679, 8643075271396760825, 7891176514265388517, 2236426588013702123]]]
proof boolean:  1.0
proof result 1 : 52.25
verified
